In [0]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import  OneHotEncoder,StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [0]:
df=pd.read_excel("TrainingDataWithEPO_Others_Applicants.xlsx")

In [0]:
df['Disposal-Reason']=df['Disposal-Reason'].map({'FULL':'Others','EARLY PAYOUT':'EARLY PAYOUT','SETTLE':'Others','PAY':'Others','LOSTCASE':'Others','DECEASED':'Others','FINANCIAL':'Others','COLLECTION':'Others','REPOSSESSION':'Others'})

In [0]:
df['Disposal-Reason']=df['Disposal-Reason'].map({'EARLY PAYOUT':'EPO','Others':'Others'})


In [0]:
indexes=np.where(df['Disposal-Reason'].isna())
ind=list(indexes)
y=df['Disposal-Reason']

X=df[["Age",
"AppNo",
"App_Prov"]]


In [0]:
print(X.shape)
print(y.shape)

In [0]:
numeric_columns=X.select_dtypes(include=["int64","float64"]).columns
numeric_columns

In [0]:
categorical_columns=X.select_dtypes(include=["object"]).columns
categorical_columns

In [0]:
numeric_transformer=Pipeline(steps=[("imputer",SimpleImputer(strategy="median")),("scalar",StandardScaler())])
categorical_transformer=Pipeline(steps=[("imputer",SimpleImputer(strategy="most_frequent")),("encoding",OneHotEncoder(handle_unknown='ignore'))])

In [0]:
preprocess=ColumnTransformer(transformers=[("num",numeric_transformer,numeric_columns),("cat",categorical_transformer,categorical_columns)])

In [0]:
pipe=Pipeline(steps=[("preprocess",preprocess),("estimator",RandomForestClassifier(25))])


In [0]:
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=25,stratify=y)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)
print(y_test[:5])

In [0]:
pipe.fit(x_train,y_train)

In [0]:
pred=pipe.predict_proba(x_test.tail(5))
print("The predicted probabilities of each class",pred)


In [0]:
predicted=pipe.predict(x_test.tail(10))
print(predicted)
x_test.tail(5).index


In [0]:
y_test[[230, 352, 722, 434, 622]]

In [0]:
predAll=pipe.predict(x_test)
accuracy_score(y_test,predAll)


In [0]:
df1=pd.read_excel("TestingDataWithSomeActive_EPO_Applicants.xlsx")

In [0]:
df1['Disposal-Reason'].value_counts()

In [0]:
df1.columns

In [0]:
testData= df1[["Age",
"AppNo",
"App_Prov"]]

In [0]:
testData.columns

In [0]:
testData10Columns=testData.loc[10:]
testData10Columns

In [0]:
prediction=pipe.predict_proba(testData10Columns)
print(prediction)


In [0]:
predicted=pipe.predict(testData10Columns)
print(predicted)

In [0]:
testData10Columns['Disposal-Reason']=predicted

In [0]:
testData10Columns.index

In [0]:
testData10Columns.to_excel(r'Pred.xlsx', index=False,header=True)